In [1]:
import pandas as pd
from pathlib import Path

In [2]:
!pip install imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 59.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.6.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
from tqdm import tqdm
import pandas as pd
import cv2
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.utils.data_utils import Sequence
from imblearn.over_sampling import RandomOverSampler
from imblearn.keras import balanced_batch_generator
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline


2025-08-19 03:32:23.657121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755574343.671239       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755574343.675628       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-19 03:32:23.694281: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'keras.utils.data_utils'

In [4]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
train.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
3,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
4,ISIC_0074311,IP_2950485,female,40.0,lower extremity,unknown,benign,0


In [5]:
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
test.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052212,IP_2842074,female,50.0,lower extremity
1,ISIC_0076545,IP_9802602,male,55.0,upper extremity
2,ISIC_0085172,IP_1705144,female,50.0,lower extremity
3,ISIC_0086709,IP_4109313,male,30.0,torso
4,ISIC_0109568,IP_0825081,male,80.0,lower extremity


In [6]:
# malignant = test[test['benign_malignant']=='malignant']
# malignant_vals = malignant['image_name'].values
# type(malignant_vals)
# len(malignant_vals)


In [7]:
malignant = train[train['benign_malignant']=='malignant']
malignant_vals = malignant['image_name'].values
type(malignant_vals)
len(malignant_vals)


513

In [8]:
malignant_labels = malignant['target'].values
malignant_labels_rand = [np.random.choice(malignant_labels) for i in range(584)]


In [9]:
malignant_list = list(malignant_vals)
type(malignant_list)
len(malignant_list)
# malignant_list
#malignant_rand = [np.random.choice(malignant_vals + '.jpg') for i in range(584)]


513

In [10]:
# DIR = '../input/siim-isic-melanoma-classification/jpeg/train'
# benign_images = []

# for i in malignant_rand:
#     image1 = plt.imread(os.path.join(DIR,i))
#     image1 = cv2.resize(image1,(224,224))
#     benign_images.append(image1)
# #     np.append(k_be,image1)


In [11]:
benign = train[train['benign_malignant']=='benign']
benign_vals = benign['image_name'].values
type(benign_vals)
len(benign_vals)


28471

In [12]:
benign_labels = benign['target'].values
benign_labels_rand = [np.random.choice(benign_labels) for i in range(584)]


In [13]:
benign_list = list(benign_vals)
type(benign_list)
# benign_rand = [np.random.choice(benign_vals + '.jpg') for i in range(584)]


list

In [14]:
benign_list_1 = benign_list[0:584]
len(benign_list_1)


584

In [15]:
mal_bin_cmbo_img_name_list = malignant_list + benign_list_1
mal_bin_cmbo_label_list = malignant_labels_rand + benign_labels_rand
print(len(mal_bin_cmbo_label_list))
print(len(mal_bin_cmbo_img_name_list))


1168
1097


In [16]:
# SEED = 42
# from sklearn.model_selection import train_test_split
# x_train,x_val,y_train,y_val = train_test_split(mal_bin_cmbo_img_name_list,mal_bin_cmbo_label_list,test_size = 0.2,random_state = SEED)


In [17]:
import gc
gc.collect()


30

In [18]:
DIR = '../input/siim-isic-melanoma-classification/jpeg/train'
img_data = []
jpeg = '.jpg'
img_size = 224

for i in tqdm(mal_bin_cmbo_img_name_list):
    img = plt.imread(os.path.join(DIR,i)+jpeg)
    img = cv2.resize(img,(img_size,img_size))
    img_data.append(img)


100%|██████████| 1097/1097 [01:17<00:00, 14.23it/s]


In [19]:
train_imgs = np.asarray(img_data)
print(train_imgs.shape)
train_labels =  np.asarray(mal_bin_cmbo_label_list)
# test_imgs = np.asarray(test_imgs)
# test_labels =  np.asarray(test_labels)


(1097, 224, 224, 3)


In [20]:
BATCH_SIZE = 64
SEED = 42
EPOCHS = 100
x, y, z = 224, 224, 3
inputShape = (x, y, z)
NUM_CLASSES = 1


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(
    train_imgs, 
    train_labels,
    test_size = 0.3, 
    random_state = SEED
)


ValueError: Found input variables with inconsistent numbers of samples: [1097, 1168]

In [22]:
type(X_val)


NameError: name 'X_val' is not defined

In [23]:
train_datagen = ImageDataGenerator( rescale = 1./255,
                                    rotation_range=90,
                                    width_shift_range=0.15,
                                    height_shift_range=0.15,
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    zoom_range=(0.9,1),
                                    fill_mode= 'nearest',
                                    brightness_range=(0.8, 1.2),
                                  )

train_generator = train_datagen.flow(X_train, Y_train, batch_size = BATCH_SIZE)
val_generator = train_datagen.flow(X_val, Y_val, batch_size = BATCH_SIZE, shuffle = True)


NameError: name 'ImageDataGenerator' is not defined

In [24]:
import matplotlib.pyplot as plt
from glob import glob
from skimage.io import imread
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
import keras
import numpy as np

from keras.models import Sequential
from keras.layers import (  Dense,
                            Flatten,
                            LeakyReLU
                         )
from keras.applications import  VGG19 
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from skimage.transform import resize
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator


ModuleNotFoundError: No module named 'keras.utils.np_utils'

In [25]:
from keras.callbacks import ModelCheckpoint
filepath = "../working/saved_models-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor = 'val_acc',verbose = 1,save_best_only = True,mode = 'max')
callbacks_list = [checkpoint]


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=../working/saved_models-improvement-{epoch:02d}-{val_acc:.2f}.hdf5

In [26]:
model = Sequential()
model.add(VGG19(include_top=False, weights='imagenet', input_shape= inputShape))
model.add(Flatten())
model.add(Dense(32))
model.add(LeakyReLU(0.001))
model.add(Dense(16))
model.add(LeakyReLU(0.001))
model.add(Dense(1, activation='sigmoid'))
model.layers[0].trainable = False

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['acc'])

History = model.fit_generator(train_generator,
    steps_per_epoch = len(X_train) // BATCH_SIZE,
    epochs = EPOCHS, 
    validation_data = val_generator,
    validation_steps = len(X_val) // BATCH_SIZE,
    callbacks = callbacks_list
)


model.save('melanoma-classification-model.h5')


2025-08-19 03:33:46.917945: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


AttributeError: 'Sequential' object has no attribute 'fit_generator'

In [27]:
# model.save('melanoma-classification-model.h5')


In [28]:
def hist(History):
    fig, ax = plt.subplots(1, 2, figsize=(20, 5))

    ax[0].plot(History.history['loss'])
    ax[0].plot(History.history['val_loss'])
    ax[0].legend(['Training loss', 'Validation Loss'],fontsize=18)
    ax[0].set_xlabel('Epochs ',fontsize=16)
    ax[0].set_ylabel('Loss',fontsize=16)
    ax[0].set_title('Training loss x Validation Loss',fontsize=16)


    ax[1].plot(History.history['acc'])
    ax[1].plot(History.history['val_acc'])
    ax[1].legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
    ax[1].set_xlabel('Epochs ',fontsize=16)
    ax[1].set_ylabel('Accuracy',fontsize=16)
    ax[1].set_title('Training Accuracy x Validation Accuracy',fontsize=16)


In [29]:
def plot_any(arr, title = ''):
    plt.figure(figsize = (15, 25))
    for i in range(len(arr)):
        plt.subplot(1,len(arr),i + 1)
        plt.title(title)
        plt.imshow(arr[i], cmap = 'gray');


In [30]:
loss, accu = model.evaluate(X_val, Y_val)
print("%s: %.2f%%" % ('Accuracy...', accu))
print("%s: %.2f" % ('loss.......', loss))


NameError: name 'X_val' is not defined

In [31]:
hist(History)


NameError: name 'History' is not defined

In [32]:
from keras.models import load_model
previous_model = load_model('../working/melanoma-classification-model.h5')


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '../working/melanoma-classification-model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [33]:
testimg_dir = test['image_name'].values
testimg_dir = list(testimg_dir)
# testimg_dir


In [34]:
submission=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')
submission.head()


,image_name,target
0,ISIC_0052212,0
1,ISIC_0076545,0
2,ISIC_0085172,0
3,ISIC_0086709,0
4,ISIC_0109568,0


In [35]:
DIR1 = '../input/siim-isic-melanoma-classification/jpeg/test'
target=[]
for i in tqdm(testimg_dir):
    img=cv2.imread(os.path.join(DIR1,i)+jpeg)
    img = cv2.resize(img, (224,224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.
    img=np.reshape(img,(1,224,224,3))
    prediction=previous_model.predict(img)
    target.append(prediction[0][0])

submission['target']=target


  0%|          | 0/4142 [00:00<?, ?it/s]


NameError: name 'previous_model' is not defined

In [36]:
submission.to_csv('submission.csv', index=False)
submission.head()


,image_name,target
0,ISIC_0052212,0
1,ISIC_0076545,0
2,ISIC_0085172,0
3,ISIC_0086709,0
4,ISIC_0109568,0


In [37]:
# !pip install Keras


In [38]:
# from keras.layers import Input,Dense,Flatten
# from keras.models import Model
# from keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import preprocess_input
# from keras.preprocessing import image
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential
# from glob import glob


In [39]:
# vgg = VGG16(input_shape = inputShape,weights = 'imagenet',include_top = False)


In [40]:
# base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')


In [41]:
# from skimage.io import imread
# from skimage.color import rgb2gray
# from skimage.transform import resize

# train_imgs = []
# train_labels = []
# test_imgs  = []
# test_labels = []

# x, y, z = 224, 224, 3

# #train

# for i in malignant_rand:
#     img_resize = imread(os.path.join(DIR,i))
#     img_resize = resize(img_resize, (x, y, z))

#     train_imgs.append(img_resize)
# #     train_labels.append(klass)
        
# #test
# # for klass, folder in enumerate(tqdm(test_path)):
# #     for img in glob(folder + '/*'):
        
# #         img_resize = imread(img)
# #         img_resize = resize(img_resize, (x, y, z))

# #         test_imgs.append(img_resize)
# #         test_labels.append(klass)


In [42]:
# train_imgs[0].shape


In [43]:
# train_imgs = np.asarray(train_imgs)


In [44]:
# train_imgs.shape


In [45]:
#I can take two list of benign_rand and malignent_rand and concat them then do the for loop for reading the images and store into a new list
#then convert that list into a nparray
# then follow 'https://www.kaggle.com/xwalker/simple-cnn-vgg19-keras'
# NUM_CLASSES = 1
# EPOCHS = 15
# BATCH_SIZE = 64
# inputShape = (x, y, z)
# X_train, X_test, y_train, y_test = train_test_split(
#     train_imgs, 
#     train_labels,
#     test_size = 0.3, 
# )
# train_datagen = ImageDataGenerator( rescale = 1./255,
#                                     rotation_range=90,
#                                     width_shift_range=0.15,
#                                     height_shift_range=0.15,
#                                     horizontal_flip = True,
#                                     vertical_flip = True,
#                                     zoom_range=(0.9,1),
#                                     fill_mode= 'nearest',
#                                     brightness_range=(0.8, 1.2),
#                                   )

# train_generator = train_datagen.flow(X_train, y_train, batch_size = BATCH_SIZE)
# val_generator = train_datagen.flow(X_test, y_test, batch_size = BATCH_SIZE, shuffle = True)


In [46]:
# concated_be_mal = benign_rand + malignant_rand
# concated_be_mal_1 = [np.random.choice(concated_be_mal)]


In [47]:
# plt.imshow(benign_images[9])


In [48]:
# for i in benign_images:
#     benign_images_1 = np.asarray(i)
    


In [49]:
# plt.imshow(benign_images_1[9])


In [50]:
# benign_images1[0]


In [51]:
# from keras.preprocessing.image import image to array


In [52]:
# from tqdm import tqdm

# DIR = '../input/siim-isic-melanoma-classification/jpeg/train'
# jpeg = '.jpg'
# labels = []
# images = []


# directory = []
# for i in train['image_name']:
#     directory.append(os.path.join(DIR,i)+jpeg)

# for i in range(train.shape[0]):
#     labels.append(train['target'])
    
# for i in tqdm(range(0,train.shape[0],32)):
#     imag = plt.imread(directory[i])
#     imag = imag/255.0
#     imag = cv2.resize(imag,(256,256))
#     images.append(imag)


In [53]:
# img_data = np.reshape(images,(images.shape[0],img_size,img_size,3))
# label_data = np.array(labels)


In [54]:
# plt.imshow(images[64])


In [55]:
# 1036*32


In [56]:
# directory[9]


In [57]:
# im = cv2.imread(directory[9])
# im = cv2.resize(im,(256,256))
# im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
# plt.imshow(im)
# print(im.shape)


In [58]:
# import gc
# gc.collect()


In [59]:
# class BalancedDataGenerator(Sequence):
#     """ImageDataGenerator + RandomOversampling"""
#     def __init__(self, x, y, datagen, batch_size=32):
#         self.datagen = datagen
#         self.batch_size = batch_size
#         self._shape = x.shape        
#         datagen.fit(x)
#         self.gen, self.steps_per_epoch = balanced_batch_generator(x.reshape(x.shape[0], -1), y, sampler=RandomOverSampler(), batch_size=self.batch_size, keep_sparse=True)

#     def __len__(self):
#         return self._shape[0] // self.batch_size

#     def __getitem__(self, idx):
#         x_batch, y_batch = self.gen.__next__()
#         x_batch = x_batch.reshape(-1, *self._shape[1:])
#         return self.datagen.flow(x_batch, y_batch, batch_size=self.batch_size).next()


In [60]:
# from keras.preprocessing.Image import ImageDataGenerator
# datagen = ImageDataGenerator() # define your data augmentation
# bgen = BalancedDataGenerator(x, y, datagen, batch_size=32)
# steps_per_epoch = balanced_gen.steps_per_epoch


In [61]:
# import gc
# gc.collect()
